# scPOST quick-start tutorial

This tutorial will run through the scPOST framework, which comprises 3 steps.

Step 1: Parameter estimation

Step 2: Dataset simulation

Step 3: Association testing for cell state frequency shifts (e.g. expansion or deletion)

Here, we will simulate fibroblast data from the rheumatoid arthritis (RA) dataset described in Zhang F, Wei K, Slowikowski K, Fonseka C, Rao DA, *et al.*, *Nature Immunol* (2020)

Through cytometry, the authors identified an HLA-DRA+ fibroblast cell state that was expanded in inflammatory RA samples compared to osteoarthritis samples. However, they we unable to detect this expansion in their single-cell RNA sequencing (scRNA-seq) data due to sample size (n = 21, 17 case, 4 control).

Let's try and use scPOST to pinpoint some changes to the study design that might improve power.

In [1]:
library(devtools)
devtools::install_github(repo = "immunogenomics/scpost", force = TRUE)

Loading required package: usethis



✔  checking for file ‘/tmp/Rtmp2n7Q4l/remotes4887690fa4b3/immunogenomics-scpost-baa74c3/DESCRIPTION’
─  preparing ‘scpost’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  looking to see if a ‘data/datalist’ file should be added
─  building ‘scpost_1.0.tar.gz’
   


In [2]:
library(scpost)

Registered S3 method overwritten by 'spatstat':
  method     from
  print.boxx cli 


# Step 1: Parameter estimation

To simulate data, we estimate two general aspects of variation often found in multi-sample single-cell data:

1. How much a cell state's frequency varies across samples (Cell state frequency variation)
1. How much a cell state's gene expression varies. We estimate and simulate gene expression with principal components (PCs), because PCs are a summary of gene expression that also takes into account gene covariation.

We estimate from real data. We offer the RA fibroblast metadata and PC embeddings as a pre-loaded object: ra_FibObj

In [3]:
ra_FibObj$meta %>% str
ra_FibObj$embeddings %>% head(2)

'data.frame':	1844 obs. of  11 variables:
 $ UMAP1       : num  -4.81 -4.13 -0.23 -4.86 1.44 ...
 $ UMAP2       : num  0.961 0.804 -2.206 0.906 1.474 ...
 $ CellID      : chr  "S006_L1Q1_M01" "S006_L1Q1_M03" "S006_L1Q1_M05" "S006_L1Q1_M07" ...
 $ sample      : Factor w/ 21 levels "300-0122","300-0153",..: 18 18 18 18 18 18 18 18 18 18 ...
 $ batch       : Factor w/ 24 levels "300-0122","300-0153",..: 18 18 18 18 18 18 18 18 18 18 ...
 $ disease     : chr  "OA" "OA" "OA" "OA" ...
 $ nUMI        : int  14660 12703 10968 16464 13746 14744 15939 7979 19584 11007 ...
 $ nGenes      : int  4152 3923 3760 4552 3734 4151 4204 2888 4714 3681 ...
 $ celltype    : Factor w/ 5 levels "B cell","Empty",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ clusAllCells: Factor w/ 12 levels "0","1","2","3",..: 6 6 7 6 4 6 6 9 6 6 ...
 $ clusOnlyFib : Factor w/ 9 levels "0","1","2","3",..: 3 3 2 3 1 3 3 1 3 3 ...


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
S006_L1Q1_M01,19.54967,-1.811336,1.338648,-3.845058,-1.56015,-1.102576,1.6973464,-0.6949008,-0.732699,-1.1053434,1.072299,-1.171199,2.041257,-0.9153473,0.6063470,-1.1541958,-2.90230,-0.71679693,-1.075425,1.4423108
S006_L1Q1_M03,16.15553,-1.055468,2.950489,1.226705,-1.64750,1.241454,0.6577186,3.2392051,-1.171088,-0.4900129,2.818605,-2.738194,3.586889,-1.8724081,-0.5646049,-0.8233941,-1.54052,-0.03015161,-1.837483,0.2765889


## Cell state frequency Variation

We estimate each cell state's mean frequency and covariance across samples

In [4]:
system.time({
    raFib_freqEstimates <- estimateFreqVar(meta = ra_FibObj$meta, clusCol = 'clusOnlyFib', sampleCol = 'sample', logCov = TRUE)
})

   user  system elapsed 
  0.208   0.000   0.208 

In [5]:
raFib_freqEstimates %>% str

List of 2
 $ meanFreq: Named num [1:9] 0.1871 0.1783 0.1481 0.1038 0.0966 ...
  ..- attr(*, "names")= chr [1:9] "clus0" "clus1" "clus2" "clus3" ...
 $ cfcov   : num [1:9, 1:9] 2.87 -1.225 0.278 0.311 -0.705 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:9] "clus0" "clus1" "clus2" "clus3" ...
  .. ..$ : chr [1:9] "clus0" "clus1" "clus2" "clus3" ...


## Gene expression variation

We use linear mixed effects models to estimate a cell state's variation in PC space. From these models, we estimate the following for each cell state:

1. Centroid in PC space
1. Batch-associated variance in PC space
1. Sample-associated variance in PC space
1. Residual variance in PC space

Computation time will depend on the size of the real data. For larger datasets, we recommend saving the aforementioned estimates so that they can be used for future simulations

In [9]:
system.time({
    raFib_pcEstimates <- estimatePCVar(pca = ra_FibObj$embeddings, npcs = 20, meta = ra_FibObj$meta, clusCol = 'clusOnlyFib',
                                       sampleCol = 'sample', batchCol = 'batch')
})

   user  system elapsed 
  6.528   0.001   6.538 

In [10]:
raFib_pcEstimates %>% str(1)
# Save raFib_pcaEstimates for future use

List of 4
 $ centroids  : num [1:9, 1:20] -1.32 -1.58 14.28 -6.91 -7.64 ...
  ..- attr(*, "dimnames")=List of 2
 $ pc_cov_list:List of 9
 $ batch_vars : num [1:9, 1:20] 0.848 5.89 4.168 5.455 1.944 ...
  ..- attr(*, "dimnames")=List of 2
 $ sample_vars: num [1:9, 1:20] 1.902757 8.78203 0.104539 0.000455 2.816822 ...
  ..- attr(*, "dimnames")=List of 2


# Step 2 and 3: Simulate dataset and perform association testing

Next, we'll generate *in silico* fibroblast datasets and use MASC (Mixed effects association of single-cells) to test whether we test an expansion or depletion of a cluster in our simulated data.

While we give the option perform **Step 2** and **Step 3** independently (especially for those that want to use a different association test) with simDataset.base, we recommend users that want to use MASC to perform both **Step 2 and Step 3** at the same time so that they may make use of streamlined file naming and handling. 

## Simulate datasets with 20 samples: unbalanced study design

The cohort of the original RA scRNA-seq data consisted of 17 cases and 4 controls. Here, we'll simulate multiple datasets with 20 samples that have 17 cases and 3 controls.

We simulate realistic levels of cell state frequency variation and gene expression variation. In case samples, we induce a fold change of 5 in "clus0", the HLA-DRA+ fibroblast cell state. 

The "simDataset.MASC" function will simulate datasets and save them as a list containing metadata information and p-values from using MASC. Before simulating data, create a folder that will contain the saved data

In [41]:
# Set up parameter table for multiple simulations
set.seed(23)

# Set number of samples, number of cells per sample, and create batch structure
ncases <- 17
nctrls <- 3
nbatches <- 4
batchStructure <- distribSamples(ncases = ncases, nctrls = nctrls, nbatches = nbatches)
ncells <- rep(250, times = ncases + nctrls)
names(ncells) <- batchStructure$sample_names

# Retrieve estimates
meanFreqs <- raFib_freqEstimates$meanFreq
cfcov <- raFib_freqEstimates$cfcov
centroids <- raFib_pcEstimates$centroids
pc_cov_list <- raFib_pcEstimates$pc_cov_list
batch_vars <- raFib_pcEstimates$batch_vars
sample_vars <- raFib_pcEstimates$sample_vars

# Set scaling parameters (realistic)
b_scale <- 1
s_scale <- 1
cf_scale <- 1

# Set cluster names to induce fc, magnitude of fc, and resolution to re-cluster
clus <- "clus0"
fc <- 5
resolution <- 0.6
mc.cores <- 1

save_path <- file.path(getwd(), "scpostSims/unbalanced/")
reps <- 1:5

# Create parameter table for running multiple simulations
params <- expand.grid(
    rep = reps, ncases = ncases, nctrls = nctrls, nbatches = nbatches, b_scale = b_scale, s_scale = s_scale,
    cf_scale = cf_scale, clus = clus, fc = fc, res_use = resolution, save_path = save_path
)
params$cond_induce = "cases"
params$seed <- sample(.Machine$integer.max, size = nrow(params))

params %>% dim
params %>% head(2)

[1]  5 13

rep,ncases,nctrls,nbatches,b_scale,s_scale,cf_scale,clus,fc,res_use,save_path,cond_induce,seed
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<chr>,<int>
1,17,3,4,1,1,1,clus0,5,0.6,/data/srlab1/nmillard/scpostSims/unbalanced/,cases,943762698
2,17,3,4,1,1,1,clus0,5,0.6,/data/srlab1/nmillard/scpostSims/unbalanced/,cases,2040394625


In [42]:
suppressWarnings({
    lapply(seq(nrow(params)), function(x){
            simDataset.withMASC(
                save_path = params[x, 'save_path'],
                rep = params[x, 'rep'],
                seed = params[x, 'seed'],
                ncases = params[x, 'ncases'],
                nctrls = params[x, 'nctrls'],
                nbatches = params[x, 'nbatches'],
                batchStructure = batchStructure,
                ncells = ncells,
                centroids = centroids,
                pc_cov_list = pc_cov_list,
                batch_vars = batch_vars,
                b_scale = params[x, 'b_scale'],
                sample_vars = sample_vars,
                s_scale = params[x, 's_scale'],
                cfcov = cfcov,
                cf_scale = params[x, 'cf_scale'],
                meanFreqs = meanFreqs,
                clus = params[x, 'clus'],
                fc = params[x, 'fc'],
                cond_induce = params[x, 'cond_induce'],
                res_use = params[x, 'res_use'], 
                mc.cores = 1,
                clusterData = TRUE,
                returnPCs = FALSE
            )
    })
})

Simulated dataset at 2020-11-20 15:43:18
Simulated dataset at 2020-11-20 15:43:25
Simulated dataset at 2020-11-20 15:43:33
Simulated dataset at 2020-11-20 15:43:39
Simulated dataset at 2020-11-20 15:43:45


[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

## Simulate datasets with 20 samples: balanced study design

Now, let's simulate 20-sample datasets with 10 cases and 10 controls

In [43]:
# Set up parameter table for multiple simulations
set.seed(23)

# Set number of samples, number of cells per sample, and create batch structure
ncases <- 10
nctrls <- 10
nbatches <- 4
batchStructure <- distribSamples(ncases = ncases, nctrls = nctrls, nbatches = nbatches)
ncells <- rep(250, times = ncases + nctrls)
names(ncells) <- batchStructure$sample_names

# Retrieve estimates
meanFreqs <- raFib_freqEstimates$meanFreq
cfcov <- raFib_freqEstimates$cfcov
centroids <- raFib_pcEstimates$centroids
pc_cov_list <- raFib_pcEstimates$pc_cov_list
batch_vars <- raFib_pcEstimates$batch_vars
sample_vars <- raFib_pcEstimates$sample_vars

# Set scaling parameters (realistic)
b_scale <- 1
s_scale <- 1
cf_scale <- 1

# Set cluster names to induce fc, magnitude of fc, and resolution to re-cluster
clus <- "clus0"
fc <- 5
resolution <- 0.6
mc.cores <- 1

save_path <- file.path(getwd(), "scpostSims/balanced/")
reps <- 1:5

# Create parameter table for running multiple simulations
params <- expand.grid(
    rep = reps, ncases = ncases, nctrls = nctrls, nbatches = nbatches, b_scale = b_scale, s_scale = s_scale,
    cf_scale = cf_scale, clus = clus, fc = fc, res_use = resolution, save_path = save_path
)
params$cond_induce = "cases"
params$seed <- sample(.Machine$integer.max, size = nrow(params))

params %>% dim
params %>% head(2)

[1]  5 13

rep,ncases,nctrls,nbatches,b_scale,s_scale,cf_scale,clus,fc,res_use,save_path,cond_induce,seed
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<chr>,<int>
1,10,10,4,1,1,1,clus0,5,0.6,/data/srlab1/nmillard/scpostSims/balanced/,cases,265763406
2,10,10,4,1,1,1,clus0,5,0.6,/data/srlab1/nmillard/scpostSims/balanced/,cases,1113800433


In [44]:
suppressWarnings({
    lapply(seq(nrow(params)), function(x){
            simDataset.withMASC(
                save_path = params[x, 'save_path'],
                rep = params[x, 'rep'],
                seed = params[x, 'seed'],
                ncases = params[x, 'ncases'],
                nctrls = params[x, 'nctrls'],
                nbatches = params[x, 'nbatches'],
                batchStructure = batchStructure,
                ncells = ncells,
                centroids = centroids,
                pc_cov_list = pc_cov_list,
                batch_vars = batch_vars,
                b_scale = params[x, 'b_scale'],
                sample_vars = sample_vars,
                s_scale = params[x, 's_scale'],
                cfcov = cfcov,
                cf_scale = params[x, 'cf_scale'],
                meanFreqs = meanFreqs,
                clus = params[x, 'clus'],
                fc = params[x, 'fc'],
                cond_induce = params[x, 'cond_induce'],
                res_use = params[x, 'res_use'], 
                mc.cores = 1,
                clusterData = TRUE,
                returnPCs = FALSE
            )
    })
})

Simulated dataset at 2020-11-20 15:43:51
Simulated dataset at 2020-11-20 15:43:58
Simulated dataset at 2020-11-20 15:44:05
Simulated dataset at 2020-11-20 15:44:12
Simulated dataset at 2020-11-20 15:44:19


[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

# Get Power

Now that we've simulated data and performed association testing, we can retrieve what our estimated power is. We only simulated 5 replicates for each study design; to achieve a more accurate estimate, we recommend running many replicates. Here, we load the results tables from MASC analysis (named "res" in the saved list that was created by the "simDataset.MASC" function)

In [45]:
dir <- file.path(getwd(), "scpostSims/balanced/")
bal_files <- list.files(path = dir,
                        full.names = T,
                        pattern = '*res') %>% basename
bal_tables <- lapply(bal_files, function(x){
        readRDS(file.path(dir, x))[["res"]]
})

dir <- file.path(getwd(), "scpostSims/unbalanced/")
unbal_files <- list.files(path = dir,
                        full.names = T,
                        pattern = '*res') %>% basename
unbal_tables <- lapply(unbal_files, function(x){
        readRDS(file.path(dir, x))[["res"]]
})

If you induced a fold change in multiple cell states, you can stratify the power results by cell state (instead of aggregate power over all cell states). Here, we only induced a fold change in the HLA-DRA+ fibroblast cell state, so that is not necessary.

In [47]:
power_Unbal <- getPowerFromRes(
    resFiles = unbal_files, 
    resTables = unbal_tables,
    threshold = 0.05,
    z = 1.96,
    stratByClus = FALSE
)
power_Bal <- getPowerFromRes(
    resFiles = bal_files, 
    resTables = bal_tables,
    threshold = 0.05,
    z = 1.96,
    stratByClus = FALSE
)

power_Unbal
power_Bal

ind_fc,ncases,nctrls,nsamples,ncells,bscale,sscale,cfscale,trials,masc_power,masc_power_ci
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<dbl>,<dbl>
5,17,3,20,250,1,1,1,5,0,0


ind_fc,ncases,nctrls,nsamples,ncells,bscale,sscale,cfscale,trials,masc_power,masc_power_ci
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<dbl>,<dbl>
5,10,10,20,250,1,1,1,5,0.8,0.3506155


From these small number of simulations, we estimated that the unbalanced study design would have 0% power. However, by changing the balance of cases and controls so that the study design is more balanced, we see an estimated power of 80%. This example showcases how scPOST can be used to evaluate how changing a study design might affect power.

For more accurate estimates, we recommend running higher numbers of simulations. In Millard *et al.*, we ran 500 simulations for each of these study designs, and estimated 12% power for the unbalanced study design and 60% power for the balanced study design.

# End